In [75]:
# pandas to work with dataframes
import pandas as pd
# import re to use regular expressions
import re
# import numpy
import numpy as np
# import glob to bring together multiple csv files
import glob

In [76]:
# search for csvs containing web-scraped data with the name scheme
csvs = glob.glob(r'*Craigslist_ApartmentData_Minneapolis*.csv')
# set dfs list to store each dataframe that gets read in
dfs = []
# iterate through csv files gathered in glob
for file in csvs:
    # read each csv as a dataframe with the same column names
    df1 = pd.read_csv(file, names=['address', 'square feet', 'beds_baths', 'datetime', 'square feet 2', 'latitude', 'title', 'longitude', 'map accuracy', 'price', 'url', 'post id'])
    # append dataframe to dfs list
    dfs.append(df1)

In [77]:
# concatenate dataframes in dfs together vertically
df = pd.concat(dfs, axis=0)
# check to see if address present, if it is, the boolean will be set to False
# this seems counterintuitive
df['address_check'] = [bool(re.match('nan', str(cell))) for cell in df['address']]
# sort values by address_check column
df = df.sort_values(['address_check'])
# set an index of integers equal to the length of dataframe rows
index_list = list(range(len(df.index)))
# make index series a column in dataframe
df['index'] = index_list
# set index column as index of dataframe
df = df.set_index(['index'])
# set entire df type as string, as most operations will require data type to be string
df = df.astype(str)

In [78]:
# drop any post id values that are null
postid_no_null = df.dropna(subset = ['post id'])
# create dataframe where "post id" column is null
null_id = df[df['post id'].isnull()]
# drop duplicates in null_id dataframe using the "title" column
null_id = null_id.drop_duplicates(subset=['title'], keep='first')
# drop duplicates using "post id" column in the dataframe containing no null "post id" values
postid_no_null = df.drop_duplicates(subset=['post id'], keep='first')
# concatenate vertically null_id and postid_no_null dataframes
df = pd.concat([null_id, postid_no_null])
# drop latitude, longitude, map accuracy columns 
df = df.drop(columns=['latitude', 'longitude', 'map accuracy'])

In [5]:
# extract and store all possible valid square feet or beds values across columns where they might be
# using findall function
# column to store values of square feet that exist in column
df['value_square_feet_two'] = [re.findall('\d+ft', str(cell)) for cell in df['square feet 2']]
# create column of all patterns of square feet that exist in  beds_baths column
df['value_square_feet_three'] = [re.findall('\d+ft', str(cell)) for cell in df['beds_baths']]
# get all patterns of beds that exist in "square feet 2" column
df['number_of_beds_2'] = [re.findall('\d+br', str(cell)) for cell in df['square feet 2']]
# clean values from columns to prepare them to be brought into "square feet", "price" and "beds_baths" columns
df['number_of_beds_2'] = df['number_of_beds_2'].astype(str)
df['value_square_feet_two'] = df['value_square_feet_two'].astype(str)
df['value_square_feet_three'] = df['value_square_feet_three'].astype(str)

In [6]:
# create a boolean of whether or not a beds/bath pattern exists in the "beds_baths" column
df['first_beds_boolean'] = [bool(re.search('\d+[Bb]', cell)) for cell in df['beds_baths']] 
# get boolean of whether or not beds value is present in "square feet 2" column
df['second_beds_boolean'] = [bool(re.search('\d+br', str(cell))) for cell in df['square feet 2']]
# clean value before moving
df.loc[df['second_beds_boolean'] == 1, 'number_of_beds_2'] = df['number_of_beds_2'].apply(lambda x: str(x[2:-2]))
# where value in "beds_baths" is not available, and is available in "square feet 2", 
# fill with the beds value of "square feet 2", stored in "number_of_beds_2"
df.loc[(df['second_beds_boolean'] == 1) & (df['first_beds_boolean'] == 0), 'beds_baths'] = df['number_of_beds_2']
# create a boolean of whether or not a square feet pattern exists in the "square feet" column
df['first_square_feet_boolean'] = [bool(re.search('\d+ft', str(cell))) for cell in df['square feet']]
# boolean to see if row in square feet 2 column contains square feet value
df['second_square_feet_boolean'] = [bool(re.search('\d+ft', str(cell))) for cell in df['square feet 2']]
# clean value before moving
df.loc[df['second_square_feet_boolean'] == 1, 'value_square_feet_two'] = df['value_square_feet_two'].apply(lambda x: str(x[2:-2]))
# where value in "square feet" is not available, and is available in "square feet 2", 
# fill with the square feet value of "square feet 2", stored in "value_square_feet_two"
df.loc[(df['first_square_feet_boolean'] == 0) & (df['second_square_feet_boolean'] == 1), 'square feet'] = df['value_square_feet_two']
# boolean to see if beds_baths column is True/False for square feet pattern
df['third_square_feet_boolean'] = [bool(re.search('\d+ft', str(cell))) for cell in df['beds_baths']]
# clean value before moving
df.loc[df['third_square_feet_boolean'] == 1, 'value_square_feet_three'] = df['value_square_feet_three'].apply(lambda x: str(x[2:-2]))
# for all values in beds_bath where the square feet pattern is true, 
# cell value is updated with number in square_feet_one
df.loc[(df['first_square_feet_boolean'] == 0) & (df['second_square_feet_boolean'] == 0) & (df['third_square_feet_boolean'] == 1), 'square feet'] = df['value_square_feet_three']

In [7]:
# for square feet, beds_baths and price columns, determine if appropriate values are present with boolean
# in order to fill these columns with appropriate values in future operations
df['square feet boolean'] = [bool(re.search('\d+ft', str(cell))) for cell in df['square feet']]
df['beds boolean'] = [bool(re.search('\d+[Bb]', str(cell))) for cell in df['beds_baths']]
df['price boolean'] = [bool(re.search('\d+', str(cell))) for cell in df['price']]


In [8]:
# get just digits in square feet column 
df.loc[(df['square feet boolean'] == 1), 'square feet'] = df['square feet'].apply(lambda x: x[0:-3])
# get just numerical value of beds_baths
df.loc[df['beds_baths'].str.contains(r'B'), 'beds_baths'] = df['beds_baths'].apply(lambda x: x[0])
# get just price in "price" column
df['price'] = df.price.apply(lambda x: x.replace('$', ''))

In [9]:
# identify and drop posts associated with the superbowl
df['check_superbowl'] = [bool(re.search("SUPERBOWL|Superbowl|SUPER BOWL|Super bowl|superbowl|super bowl|Super Bowl", cell)) for cell in df['title']]
df = df.drop(df[df['check_superbowl'] == 1].index)

In [10]:
# extract url information from "url" column and fill as value in "county" column
df['county'] = [str(re.findall('org/.../', cell))[6:9] for cell in df['url']]
# create county_lat, county_lon columns
df['county_state'] = df['county']
df['county_state'] = df['county']
# create a column county_state with county and state to append to each respective address
df['county_state'] = df['county']
df.loc[(df['county'] == "hnp"), "county_state"] = " ,Hennepin County, Minnesota"
df.loc[(df['county'] == "ram"), "county_state"] = " ,Ramsey County, Minnesota"
df.loc[(df['county'] == "dak"), "county_state"] = " ,Dakota County, Minnesota"
df.loc[(df['county'] == "ank"), "county_state"] = " ,Anoka County, Minnesota"
df.loc[(df['county'] == "wsh"), "county_state"] = " ,Washington County, Minnesota"
df.loc[(df['county'] == "csw"), "county_state"] = " ,Carver County, Minnesota"
# append the value in county_state to each respective address
df.loc[(df['address_check'] == True), 'address'] = df[['address', 'county_state']].apply(''.join, axis=1)

In [11]:
# create geocode_lat and geocode_lon columns for geocoding
df['geocode_lat'] = "none"
df['geocode_lon'] = "none"

In [12]:
# save dataframe without missing addresses to be geocoded
# then resume script after values have been geocoded
df.to_csv('geocode_craigslist_rentals_2.csv')